# Generate test data frame with user actions in ecommerce 

Task - For each day, calculate what percentage of product views ended in a purchase

### Generate data

In [960]:
import pandas as pd
import numpy as np
import time

In [961]:
# Generate table user_actions
#
# user_id
# product_id
# action     
# time       
# date       

In [962]:
all_user_ids = np.arange(1, 1000)
all_products_ids = np.arange(1,100)

In [963]:
n = 10000

In [964]:
# сreate df
user_ids = np.random.choice(all_user_ids, n)
product_ids = np.random.choice(all_products_ids, n)
start_date = pd.to_datetime('2022-01-01')
times = pd.date_range(start_date, periods=n, freq='1min')

In [965]:
times

DatetimeIndex(['2022-01-01 00:00:00', '2022-01-01 00:01:00',
               '2022-01-01 00:02:00', '2022-01-01 00:03:00',
               '2022-01-01 00:04:00', '2022-01-01 00:05:00',
               '2022-01-01 00:06:00', '2022-01-01 00:07:00',
               '2022-01-01 00:08:00', '2022-01-01 00:09:00',
               ...
               '2022-01-07 22:30:00', '2022-01-07 22:31:00',
               '2022-01-07 22:32:00', '2022-01-07 22:33:00',
               '2022-01-07 22:34:00', '2022-01-07 22:35:00',
               '2022-01-07 22:36:00', '2022-01-07 22:37:00',
               '2022-01-07 22:38:00', '2022-01-07 22:39:00'],
              dtype='datetime64[ns]', length=10000, freq='T')

In [924]:
user_actions = pd.DataFrame({'user_id': user_ids,
                             'product_id': product_ids,
                             'time': times })

In [925]:
user_actions

,user_id,product_id,time
0,615,96,2022-01-01 00:00:00
1,597,99,2022-01-01 00:01:00
2,857,50,2022-01-01 00:02:00
3,70,11,2022-01-01 00:03:00
4,144,70,2022-01-01 00:04:00
...,...,...,...
9995,417,25,2022-01-07 22:35:00
9996,515,92,2022-01-07 22:36:00
9997,266,53,2022-01-07 22:37:00
9998,73,52,2022-01-07 22:38:00


In [926]:
# set action view
user_actions['action'] = 'view'

In [927]:
user_actions.head()

,user_id,product_id,time,action
0,615,96,2022-01-01 00:00:00,view
1,597,99,2022-01-01 00:01:00,view
2,857,50,2022-01-01 00:02:00,view
3,70,11,2022-01-01 00:03:00,view
4,144,70,2022-01-01 00:04:00,view


In [928]:
user_actions.shape

(10000, 4)

In [929]:
#pd.DataFrame(np.random.binomial(1, 0.2, 100)).value_counts()

## Add add to cart action and purchase action

In [930]:
def generate_funel_actions(index, **kwargs):
    to_add_to_cart = 0.2 #probability to add cart
    to_purchase = 0.5 #probability by
    
    df = pd.DataFrame()
    
    if np.random.binomial(1, to_add_to_cart, 1)[0]:
        df = pd.DataFrame(kwargs, index = [0])
        df.time = df.time + np.timedelta64(5, 's')
        df.action = 'add_to_cart'
        
        if np.random.binomial(1, to_purchase, 1)[0]:
            df_purchase = pd.DataFrame(kwargs, index = [0])
            df_purchase.time = df_purchase.time + np.timedelta64(10, 's')
            df_purchase.action = 'purchase'
            df = pd.concat([df,df_purchase], ignore_index=True)

    
    return df

In [931]:
# Test our function
for _ in range(10):
    user_actions.loc[0].to_dict()
    print(generate_funel_actions(1, **user_actions.loc[0].to_dict()))

Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
   user_id  product_id                time       action
0      615          96 2022-01-01 00:00:05  add_to_cart
1      615          96 2022-01-01 00:00:10     purchase
   user_id  product_id                time       action
0      615          96 2022-01-01 00:00:05  add_to_cart
1      615          96 2022-01-01 00:00:10     purchase
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []


In [932]:
user_actions.head()

,user_id,product_id,time,action
0,615,96,2022-01-01 00:00:00,view
1,597,99,2022-01-01 00:01:00,view
2,857,50,2022-01-01 00:02:00,view
3,70,11,2022-01-01 00:03:00,view
4,144,70,2022-01-01 00:04:00,view


In [933]:
to_cart_df = pd.DataFrame()

for i, row in user_actions.iterrows():
    to_cart_df = pd.concat([to_cart_df,generate_funel_actions(i, **row)], ignore_index=True)

In [934]:
to_cart_df.head()

,user_id,product_id,time,action
0,70,11,2022-01-01 00:03:05,add_to_cart
1,901,86,2022-01-01 00:05:05,add_to_cart
2,901,86,2022-01-01 00:05:10,purchase
3,587,12,2022-01-01 00:11:05,add_to_cart
4,587,12,2022-01-01 00:11:10,purchase


In [935]:
user_actions = pd.concat([user_actions,to_cart_df])

In [936]:
user_actions.head()

,user_id,product_id,time,action
0,615,96,2022-01-01 00:00:00,view
1,597,99,2022-01-01 00:01:00,view
2,857,50,2022-01-01 00:02:00,view
3,70,11,2022-01-01 00:03:00,view
4,144,70,2022-01-01 00:04:00,view


In [937]:
user_actions = user_actions.sort_values(by=['time'])

In [938]:
user_actions.action.unique()

array(['view', 'add_to_cart', 'purchase'], dtype=object)

In [939]:
user_actions['date'] = user_actions.time.dt.date

In [940]:
user_actions.head()

,user_id,product_id,time,action,date
0,615,96,2022-01-01 00:00:00,view,2022-01-01
1,597,99,2022-01-01 00:01:00,view,2022-01-01
2,857,50,2022-01-01 00:02:00,view,2022-01-01
3,70,11,2022-01-01 00:03:00,view,2022-01-01
0,70,11,2022-01-01 00:03:05,add_to_cart,2022-01-01


## Solve task

For each day, calculate what percentage of product views ended in a purchase

### Solve with Pandas


In [941]:
report_temp = user_actions.groupby(['date','action'])['time'].count().reset_index()

In [942]:
report_temp

,date,action,time
0,2022-01-01,add_to_cart,284
1,2022-01-01,purchase,151
2,2022-01-01,view,1440
3,2022-01-02,add_to_cart,299
4,2022-01-02,purchase,167
5,2022-01-02,view,1440
6,2022-01-03,add_to_cart,294
7,2022-01-03,purchase,145
8,2022-01-03,view,1440
9,2022-01-04,add_to_cart,262


In [943]:
report = pd.pivot_table(report_temp, index=['date'], columns=['action']).reset_index()

In [944]:
report

date        time               
action             add_to_cart purchase  view
0       2022-01-01         284      151  1440
1       2022-01-02         299      167  1440
2       2022-01-03         294      145  1440
3       2022-01-04         262      123  1440
4       2022-01-05         318      151  1440
5       2022-01-06         267      113  1440
6       2022-01-07         278      143  1360

In [945]:
report.columns

MultiIndex([('date',            ''),
            ('time', 'add_to_cart'),
            ('time',    'purchase'),
            ('time',        'view')],
           names=[None, 'action'])

In [946]:
report.columns = ['_'.join(col).strip() for col in report.columns]

In [947]:
report.head()

,date_,time_add_to_cart,time_purchase,time_view
0,2022-01-01,284,151,1440
1,2022-01-02,299,167,1440
2,2022-01-03,294,145,1440
3,2022-01-04,262,123,1440
4,2022-01-05,318,151,1440


In [948]:
report['purchase_percantage'] = report.time_purchase / report.time_view

In [949]:
report

,date_,time_add_to_cart,time_purchase,time_view,purchase_percantage
0,2022-01-01,284,151,1440,0.104861
1,2022-01-02,299,167,1440,0.115972
2,2022-01-03,294,145,1440,0.100694
3,2022-01-04,262,123,1440,0.085417
4,2022-01-05,318,151,1440,0.104861
5,2022-01-06,267,113,1440,0.078472
6,2022-01-07,278,143,1360,0.105147


### Solve with SQL


In [950]:
import sqlalchemy as sq
from sqlalchemy.orm import Session
from sqlalchemy_utils import database_exists, create_database

import csv
from io import StringIO

In [951]:
engine = sq.create_engine('postgresql://jupyterdb_user:jupyterdb_user_123@db:5432/generated_data_frame_with_user_actions_in_ecommerce')
if not database_exists(engine.url):
    create_database(engine.url)

print(database_exists(engine.url), engine.url)

True postgresql://jupyterdb_user:jupyterdb_user_123@db:5432/generated_data_frame_with_user_actions_in_ecommerce


In [952]:
#select function
def select(sql: str, engine = engine) -> pd.DataFrame:
    with engine.connect() as connection:
        df_sql = pd.DataFrame(connection.execute(sq.text(sql)))
    return df_sql

In [953]:
# upload function from https://stackoverflow.com/questions/23103962/how-to-write-dataframe-to-postgres-table

def psql_insert_copy(table, conn, keys, data_iter):
    # gets a DBAPI connection that can provide a cursor
    dbapi_conn = conn.connection
    with dbapi_conn.cursor() as cur:
        s_buf = StringIO()
        writer = csv.writer(s_buf)
        writer.writerows(data_iter)
        s_buf.seek(0)

        columns = ', '.join('"{}"'.format(k) for k in keys)
        if table.schema:
            table_name = '{}.{}'.format(table.schema, table.name)
        else:
            table_name = table.name

        sql = 'COPY {} ({}) FROM STDIN WITH CSV'.format(
            table_name, columns)
        cur.copy_expert(sql=sql, file=s_buf)

In [954]:
#upload data
user_actions.to_sql('user_actions_in_ecommerce', 
                    engine, index=False,
                    if_exists='replace',
                    method=psql_insert_copy)

In [955]:
#test sql
select("select * from user_actions_in_ecommerce t limit 100")

,user_id,product_id,time,action,date
0,615,96,2022-01-01 00:00:00,view,2022-01-01
1,597,99,2022-01-01 00:01:00,view,2022-01-01
2,857,50,2022-01-01 00:02:00,view,2022-01-01
3,70,11,2022-01-01 00:03:00,view,2022-01-01
4,70,11,2022-01-01 00:03:05,add_to_cart,2022-01-01
...,...,...,...,...,...
95,757,77,2022-01-01 01:12:00,view,2022-01-01
96,460,32,2022-01-01 01:13:00,view,2022-01-01
97,652,76,2022-01-01 01:14:00,view,2022-01-01
98,840,40,2022-01-01 01:15:00,view,2022-01-01


In [956]:
sql = '''
SELECT t.date,
count(case when t.action = 'view' then 1 else NULL end) as views,
count(case when t.action = 'add_to_cart' then 1 else NULL end) as carts,
count(case when t.action = 'purchase' then 1 else NULL end) as purchases
FROM
user_actions_in_ecommerce t
GROUP BY date
'''

In [957]:
select(sql)

,date,views,carts,purchases
0,2022-01-02,1440,299,167
1,2022-01-06,1440,267,113
2,2022-01-07,1360,278,143
3,2022-01-04,1440,262,123
4,2022-01-05,1440,318,151
5,2022-01-03,1440,294,145
6,2022-01-01,1440,284,151


In [958]:
sql = '''
with report_template as (
    SELECT t.date,
    count(case when t.action = 'view' then 1 else NULL end) as views,
    count(case when t.action = 'add_to_cart' then 1 else NULL end) as carts,
    count(case when t.action = 'purchase' then 1 else NULL end) as purchases
    FROM
    user_actions_in_ecommerce t
    GROUP BY date
)
select t.date, t.purchases, t.views, t.purchases/t.views::float as purchase_percantage from report_template t
'''

In [959]:
select(sql)

,date,purchases,views,purchase_percantage
0,2022-01-02,167,1440,0.115972
1,2022-01-06,113,1440,0.078472
2,2022-01-07,143,1360,0.105147
3,2022-01-04,123,1440,0.085417
4,2022-01-05,151,1440,0.104861
5,2022-01-03,145,1440,0.100694
6,2022-01-01,151,1440,0.104861
